In [1]:
from langchain.llms import Ollama


In [2]:
model = Ollama(model="mistral")

In [4]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

#setup
API_KEY = ""
INDEX_NAME = "healthcare-chatbot"


pc = Pinecone(api_key=API_KEY)
index = pc.Index(INDEX_NAME)

sent_model = SentenceTransformer('all-MiniLM-L6-v2')


query = "what is milk allergy? what causes it"
query_vector = sent_model.encode(query).tolist()

rel_content = index.query(
    # namespace="example-namespace",
    vector=query_vector,
    top_k=3,
    include_metadata=True
)
contexts = {item['metadata']['content'] for item in rel_content['matches']}

/home/fm-pc-lt-237/miniconda3/envs/llm/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/fm-pc-lt-237/miniconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# for i in model.invoke(template):
#     print(i)

In [43]:
import ollama


def get_prompt(contexts, question):    
    template = f"""
    You are an expert Doctor/Physician chatbot. You'll be asked question regarding different diseases and health related issues. 
    Consider the given information/context below to answer the user question. Also if the question is not health related then reply with "Cannot process."
    Context:{contexts}
    Question:{question}
    Answer: 
    """
    return template

def get_relevant_results(query):
    query_vector = sent_model.encode(query).tolist()

    rel_content = index.query(
        # namespace="example-namespace",
        vector=query_vector,
        top_k=3,
        include_metadata=True
    )
    contexts = [item['metadata']['content'] for item in rel_content['matches']]
    sources = [item['metadata']['source'] for item in rel_content['matches']]
    print(" ".join(contexts))
    return " ".join(contexts), sources

def generate_response(query):
    contexts, source = get_relevant_results(query)
    prompt = get_prompt(contexts, query)
    print(source)
    res = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    return res["message"]["content"]

In [44]:
query = 'Symptoms of milk allergy'

generate_response(query)

An allergic reaction to milk usually begins within minutes of ingestion. The severity of symptoms can vary between people. Mild symptoms may include: Hives. Nausea or vomiting. Abdominal pain. Diarrhea. Rash. Itching. Tingling or swelling of your lips, tongue or throat. Severe symptoms may include anaphylaxis. Without immediate treatment, anaphylaxis may result in death. Other severe symptoms may include: Chest tightness. Shortness of breath (dyspnea). Difficulty breathing. Difficulty swallowing (dysphagia). Rash (contact dermatitis). Wheezing. Dizziness. Drop in blood pressure (hypotension). Loss of consciousness (syncope). See a healthcare provider if you regularly have milk allergy symptoms or if you notice that your symptoms develop after ingesting milk. A milk allergy is one of the most common types of food allergy. Your immune system overreacts to one or more of the proteins in milk you've ingested (eaten or drunk). Cow's milk is the most common cause of a milk allergy. However, 

' Symptoms of a milk allergy can include hives, nausea or vomiting, abdominal pain, diarrhea, rash, itching, tingling or swelling of your lips, tongue or throat. In severe cases, symptoms may also include chest tightness, shortness of breath (dyspnea), difficulty breathing, difficulty swallowing (dysphagia), rash (contact dermatitis), wheezing, dizziness, drop in blood pressure (hypotension), loss of consciousness (syncope). If you have severe allergic reaction symptoms such as trouble breathing, call 911 (or your local emergency number) or go to your nearest emergency room (ER) immediately.'

In [ ]:
 The severity of symptoms can vary between people. Mild symptoms may include: 
    Hives. Nausea or vomiting. Abdominal pain. Diarrhea. Rash. Itching. Tingling or swelling of your lips, tongue or throat. 
Severe symptoms may include anaphylaxis. Without immediate treatment, anaphylaxis may result in death. Other severe symptoms may include: Chest tightness. Shortness of breath (dyspnea). Difficulty breathing. Difficulty swallowing (dysphagia). Rash (contact dermatitis). Wheezing. Dizziness. Drop in blood pressure (hypotension). Loss of consciousness (syncope). See a healthcare provider if you regularly have milk allergy symptoms or if you notice that your symptoms develop after ingesting milk. A milk allergy is one of the most common types of food allergy. Your immune system overreacts to one or more of the proteins in milk you've ingested (eaten or drunk). Cow's milk is the most common cause of a milk allergy. However, other types of animal milk, including goat's milk and sheep's milk, may cause your immune system to react. You may have a true milk allergy as a result of an immune system reaction. There are also other reactions to foods as well, including: Milk protein intolerance in infants. Lactose intolerance in older children and adults. A milk allergy can be deadly. If you have severe allergic reaction symptoms, such as trouble breathing, call 911 (or your local emergency number) or go to your nearest emergency room (ER) immediately.
